In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder

# Load the CSV data into a DataFrame
df = pd.read_csv('data.csv')  # Replace 'your_jobs_data.csv' with the actual file path

# Encode categorical variables
label_encoder = LabelEncoder()
df['job_title_encoded'] = label_encoder.fit_transform(df['jobTitle'])
df['user_encoded'] = label_encoder.fit_transform(df['user'])  # Assuming 'user' is a user identifier in your data

# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Create a user-item matrix
user_item_matrix = train_data.pivot_table(index='user_encoded', columns='job_title_encoded', values='application', fill_value=0)

# Perform matrix factorization using Truncated SVD
n_components = 10  # You can adjust the number of components based on your data
svd = TruncatedSVD(n_components=n_components)
user_factors = svd.fit_transform(user_item_matrix)
job_factors = svd.components_.T

# Calculate cosine similarity between user and job factors
similarity_matrix = cosine_similarity(user_factors, job_factors)

# Function to get job recommendations for a given user
def get_job_recommendations(user_id, top_n=5):
    user_index = label_encoder.transform([user_id])[0]
    user_similarity = similarity_matrix[user_index]
    ranked_jobs = sorted(list(enumerate(user_similarity)), key=lambda x: x[1], reverse=True)
    recommended_jobs = [label_encoder.inverse_transform([job_index])[0] for job_index, _ in ranked_jobs[:top_n]]
    return recommended_jobs

# Example: Get job recommendations for a user
user_id = 'some_user_id'  # Replace with the actual user identifier
recommendations = get_job_recommendations(user_id)
print("Recommended Jobs:", recommendations)
